In [10]:
import pandas as pd
import datetime as dt

In [14]:
f = lambda s: dt.datetime.strptime(s,'%d-%m-%Y %H:%M:%S')
# df_bn5 = pd.read_csv("data/historical/BankNiftyFut-Sep-5min-20200908.csv", sep="\t", parse_dates=['Date Time'], date_parser=f, encoding="utf_16_LE") 
df = pd.read_csv("data/historical/BankNiftyFut-Sep-5min-20200910.csv", sep="\t", parse_dates=['Date Time'], encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [15]:
df = df[df.columns[:-1]]
df.set_index('Date Time', inplace=True)

#### Apply rolling mean columns for data

In [ ]:
today_df['SMA6'] = today_df['Close'].rolling(6).mean()
today_df['SMA21'] = today_df['Close'].rolling(21).mean()

In [ ]:
today_df_bn5.head(10)

### Strategy function

In [ ]:
def backtest_Intraday_5min_strategy(df):
    #Parameters
    STOPLOSS = 100
    SMA6_dead_band = 50
    
    #Other Variables
    SELL = False
    BUY = False
    SELL_PRICE = 0
    BUY_PRICE = 0
    GAIN_POINTS = 0
    SELL_STOPLOSS = 0
    BUY_STOPLOSS = 0
    Total_Signals = 0
    Total_Sell_Signal = 0
    Total_Buy_Signal = 0
    start_time = dt.datetime.strptime("09:20", "%H:%M").time()
    close_time = dt.datetime.strptime("15:15", "%H:%M").time()
    entry_close_time = dt.datetime.strptime("15:00", "%H:%M").time()
    print("Time\t\t\t", "Close\t\t", "SMA6\t\t", "SMA21\t\t", "SIGNAL\t", "TYPE\t\t",  "Points\t",)
    
    for row in df.itertuples():
        SMA6_upper_band = row.SMA6 + SMA6_dead_band
        SMA6_lower_band = row.SMA6 - SMA6_dead_band
        
        # SELL STOPLOSS
        if row.Index.time() >= start_time and row.Index.time() <= close_time: 
            points = 0
            if SELL and row.Close > SELL_STOPLOSS:
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Buy_Signal += 1
                points = SELL_PRICE - row.Close
                GAIN_POINTS = GAIN_POINTS + points
                SELL_PRICE = 0
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "SELL STOPLOSS\t", round(points,2))
            
            # BUY STOPLOSS
            points = 0
            if BUY and row.Close < BUY_STOPLOSS:
                points = row.Close - BUY_PRICE
                GAIN_POINTS = GAIN_POINTS + points
                BUY_PRICE = 0
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Sell_Signal += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "BUY STOPLOSS\t", round(points,2))
            
            # SELL ENTRY
            points = 0
            if row.Close < SMA6_lower_band and \
                    row.Index.time() < entry_close_time and \
                    SELL is False:
                SELL_PRICE = row.Close
                SELL_STOPLOSS = SELL_PRICE + STOPLOSS
                if BUY_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                SELL = True
                BUY = False
                Total_Sell_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL ENTRY\t", round(points,2))
             
            # BUY ENTRY
            points = 0
            if row.Close > SMA6_upper_band and \
                    row.Index.time() < entry_close_time and \
                    BUY is False:
                BUY_PRICE = row.Close
                BUY_STOPLOSS = BUY_PRICE - STOPLOSS
                if SELL_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                BUY = True
                SELL = False
                Total_Buy_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t", row.Close,  "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY ENTRY\t", round(points,2))
            
            # DAY Close
            points = 0    
            if row.Index.time() >= close_time:
                if SELL:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = SELL_PRICE - row.Close
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY DAYCLOSE\t", round(points,2))

                if BUY:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = row.Close - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL DAYCLOSE\t", round(points,2))
                    
                    
    
    print("\n\nTotal Signals:\t\t", Total_Signals)
    print("Total Sell Signals:\t", Total_Buy_Signal)
    print("Total Buy Signals:\t", Total_Sell_Signal)
    print("Total Points Gained:\t", round(GAIN_POINTS,2))

## Function to loop through dates and call strategy

## Input the evaluation dates below

In [16]:
start_date = "2020-09-08"
end_date = "2020-09-10"

In [20]:
dt_start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
dt_end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')

eval_date = dt_start_date
while eval_date <= dt_end_date:
    eval_date = eval_date.strftime('%Y-%m-%d')
    today_df = df[eval_date]
    print(eval_date)
    print(today_df.tail(3))
    eval_date = (pd.to_datetime(eval_date) + pd.DateOffset(days=1))

2020-09-08
                         Open       Low      High     Close  Volume
Date Time                                                          
2020-09-08 15:20:00  22747.30  22702.20  22748.85  22715.25  140675
2020-09-08 15:25:00  22714.95  22655.15  22720.95  22657.45  158350
2020-09-08 15:30:00  22655.00  22655.00  22655.00  22655.00     375
2020-09-09
                         Open      Low     High    Close  Volume
Date Time                                                       
2020-09-09 15:20:00  22393.00  22354.0  22393.0  22355.6  117425
2020-09-09 15:25:00  22353.05  22338.6  22377.4  22375.0  144900
2020-09-09 15:30:00  22375.00  22375.0  22375.0  22375.0      25
2020-09-10
                         Open       Low      High     Close  Volume
Date Time                                                          
2020-09-10 15:15:00  22520.80  22483.75  22532.00  22529.55   89750
2020-09-10 15:20:00  22527.15  22524.75  22545.10  22539.85  102325
2020-09-10 15:25:00  22539.80 

In [ ]:
backtest_Intraday_5min_strategy(today_df_bn5)